# Lab 11: Índices & Optimização

In [ ]:
%load_ext sql
%config SqlMagic.displaycon = 0
%config SqlMagic.displaylimit = 100
%config SqlMagic.feedback = 0
%sql postgresql+psycopg://db:db@postgres/db --alias psql
%sql sqlite:////home/jovyan/data/measurements.db --alias sqlite

## 1. Importação

### 1.1. Importação de dados em PostgreSQL

Importe os dados do ficheiro usando as instruções SQL que se seguem num Terminal (psql):

1. Ligue-se ao PostgreSQL usando o cliente de linha de comandos `psql`.

```bash
psql -h postgres -U db
```

2. Introduza a password do utilizador `db`.

    `db`

3. Garanta que a tabela `measurement` e os índices associados não existem

```sql
DROP TABLE IF EXISTS measurement;

```
   
4. Crie a tabela `measurement` com tipos de dados adequados

```sql
CREATE TABLE
  measurement (city TEXT, temperature FLOAT);

```
   
5. Importe as medições a partir do ficheiro `data/measurements.txt`

```sql
\copy measurement(city, temperature) FROM 'data/measurements.txt' DELIMITER ';' CSV;

```

### 1.2. Importação de dados em SQLite

Importe os dados do ficheiro usando as instruções SQL que se seguem num Terminal (sqlite3):

1. Abra a `measurements.db` usando o cliente de linha de comandos `sqlite3`.

```bash
sqlite3 data/measurements.db
```

2. Garanta que a tabela `measurement` e os índices associados não existem

```sql
DROP TABLE IF EXISTS measurement;

```

3. Crie a tabela `measurement` com tipos de dados adequados

```sql
CREATE TABLE
  measurement (city TEXT, temperature FLOAT);

```

4. Importe as medições a partir do ficheiro `data/measurements.txt`

```sql
.mode csv
.separator ;
.import data/measurements.txt measurement

```

Após a importação dos dados prossiga com o notebook no JupyterLab.

## 2. Optimização

### 2.1. Planeamento de Consultas e Optimização no PostgreSQL

Fazemos agora uma primeira análise do plano para a consulta que queremos optimizar no psql:

In [ ]:
%%sql psql
EXPLAIN (ANALYZE, BUFFERS)
SELECT
  city,
  MIN(temperature) min_temperature,
  CAST(AVG(temperature) AS DECIMAL(8, 1)) mean_temperature,
  MAX(temperature) max_measure
FROM
  measurement
GROUP BY
  city
ORDER BY
  city
LIMIT
  5;

Quais são as colunas envolvidas em operações com custo elevado (i.e., `cost`)?

a) Estime _Queries-Per-Second_ (QPS)

    Total_Time = Planning_Time + Execution_Time
    QPS = 1000 (ms) / Total_Time (ms)

### 2.2. Planeamento de Consultas e Optimização no SQLite

Fazemos agora uma primeira análise do plano para a consulta que queremos optimizar no sqlite:

In [ ]:
%%sql sqlite
EXPLAIN QUERY PLAN
SELECT
  city,
  MIN(temperature) min_temperature,
  CAST(AVG(temperature) AS DECIMAL(8, 1)) mean_temperature,
  MAX(temperature) max_measure
FROM
  measurement
GROUP BY
  city
ORDER BY
  city
LIMIT
  5;

Sem indicação de custo ou buffers concentrámo-nos apenas no reconhecimento das operações no plano de consulta.

Nota: A criação de índices _apriori_ poderá evitar a computação de estruturas de dados auxiliares no momento da consulta.

## 3. Criação de Índices

Ambos os planos parecem indicar que fará sentido criar um índice na coluna *city* para optimizar o GROUP BY (e o ORDER BY).

### 3.1. Criação de Índice em `city` no PostgreSQL

In [ ]:
%%sql psql
CREATE INDEX CONCURRENTLY measurement_idx_city ON measurement (city);

In [ ]:
%%sql psql
EXPLAIN (ANALYZE, BUFFERS)
SELECT
  city,
  MIN(temperature) min_temperature,
  CAST(AVG(temperature) AS DECIMAL(8, 1)) mean_temperature,
  MAX(temperature) max_measure
FROM
  measurement
GROUP BY
  city
ORDER BY
  city
LIMIT
  5;

Esta consulta é agora essencialmente um Index Scan. Ainda estamos a ler muitos dados do disco (*buffers*). Podemos fazer melhor?

Nota: Há uma redução significativa da complexidade do GROUP BY (GroupAggregate).

a) Estime _Queries-Per-Second_ (QPS)

    Total_Time = Planning_Time + Execution_Time
    QPS = 1000 (ms) / Total_Time (ms)

### 3.2. Criação de Índice em `city` no SQLite

In [ ]:
%%sql sqlite
CREATE INDEX measurement_idx_city ON measurement (city);

In [ ]:
%%sql sqlite
EXPLAIN QUERY PLAN
SELECT
  city,
  MIN(temperature) min_temperature,
  CAST(AVG(temperature) AS DECIMAL(8, 1)) mean_temperature,
  MAX(temperature) max_measure
FROM
  measurement
GROUP BY
  city
ORDER BY
  city
LIMIT
  5;

Esta consulta é agora um Index Scan. Podemos fazer melhor?

## 4. Criação de Índices Compostos

Podemos criar um *composite index* que cubra também a *temperature*!

### 4.1 Criação de Índice Composto por `(city, temperature)` no PostgreSQL

In [ ]:
%%sql psql
CREATE INDEX CONCURRENTLY measurement_idx_city_temperature ON measurement (city, temperature);

In [ ]:
%%sql psql
EXPLAIN (ANALYZE, BUFFERS)
SELECT
  city,
  MIN(temperature) min_temperature,
  CAST(AVG(temperature) AS DECIMAL(8, 1)) mean_temperature,
  MAX(temperature) max_measure
FROM
  measurement
GROUP BY
  city
ORDER BY
  city
LIMIT
  5;

Esta consulta é agora essencialmente um Index Only Scan.

Nota: Há uma redução significativa do acesso ao disco (*buffers*).

a) Estime _Queries-Per-Second_ (QPS)

    Total_Time = Planning_Time + Execution_Time
    QPS = 1000 (ms) / Total_Time (ms)

### 4.2. Criação de Índice Composto por `(city, temperature)` no SQLite

In [ ]:
%%sql sqlite
CREATE INDEX measurement_idx_city_temperature ON measurement (city, temperature);

In [ ]:
%%sql sqlite
EXPLAIN QUERY PLAN
SELECT
  city,
  MIN(temperature) min_temperature,
  CAST(AVG(temperature) AS DECIMAL(8, 1)) mean_temperature,
  MAX(temperature) max_measure
FROM
  measurement
GROUP BY
  city
ORDER BY
  city
LIMIT
  5;

Nota: A expressão equivalente a Index Only Scan no sqlite é SCAN attrs USING COVERING INDEX.

## 5. Criação de Índices no _Schema_

Agora que já sabemos que o *composite index* é útil para a consulta que estamos a optimizar, podemos criá-lo como parte do processo de criação da tabela.

No entanto, note que se fizermos a importação dos dados após a criação do *composite index*, a importação irá demorar mais tempo devido à computação do índice em simultâneo. 

### 5.1. Criação de Índices no _Schema_ em PostgreSQL

Note a diferença, importando novamente os dados do ficheiro usando as instruções SQL que se seguem no Terminal (psql):

1. Garanta que a tabela `measurement` e os índices associados não existem

```sql
DROP TABLE IF EXISTS measurement;

```
   
2. Crie a tabela `measurement` com tipos de dados adequados

```sql
CREATE TABLE
  measurement (city TEXT, temperature FLOAT);

```

3. Crie o índice adequado para a consulta

```sql
CREATE INDEX CONCURRENTLY measurement_idx_city_temperature ON measurement (city, temperature);

```

4. Importe as medições a partir do ficheiro `data/measurements.txt`

```sql
\copy measurement(city, temperature) FROM 'data/measurements.txt' DELIMITER ';' CSV;

```

### 5.2. Criação de Índices no _Schema_ em SQLite

Note a diferença, importando novamente os dados do ficheiro usando as instruções SQL que se seguem no Terminal (sqlite3):

1. Garanta que a tabela `measurement` e os índices associados não existem

```sql
DROP TABLE IF EXISTS measurement;

```
   
2. Crie a tabela `measurement` com tipos de dados adequados

```sql
CREATE TABLE
  measurement (city TEXT, temperature FLOAT);

```

3. Crie o índice adequado para a consulta

```sql
CREATE INDEX measurement_idx_city_temperature ON measurement (city, temperature);

```

4. Importe as medições a partir do ficheiro `data/measurements.txt`

```sql
.mode csv
.separator ;
.import data/measurements.txt measurement

```